In [5]:
!pip install pandas sklearn gym

In [8]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from datetime import datetime
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import gym
import numpy as np

In [ ]:
class LineAgent:
    def __init__(self):
        self.vectors = np.zeros(shape=(2, 10))

    def predict(self, actions):
        self.vectors = self.vectors + actions


In [ ]:
from operator import truediv


class VectorEnv(gym.Env):
    def __init__(self, episodes):
        super(VectorEnv, self).__init__()
        self.action_space = gym.spaces.Box(low=-5, high=5, shape=(2, 10))
        self.observation_space = (4,38)
        self.episodes = episodes
        self.reset()

    def step(self, action):
        self.t += 1
        
        if self.t > 100:
            self.done = True
        else:
            self.done = False

        self.state = self.state + action

        # Calculate reward based on state and episode

    def reset(self):
        self.state = np.zeros(shape=(2, 10))
        self.episode = self.episodes.sample()
        self.t = 0
        return self.state

        

In [3]:
dataset = pd.read_csv(os.path.join(os.getcwd(), "../../data/vectors/vector.csv"), header = None, delimiter=',', names = list(range(150)))

In [4]:
dataset.dropna(how='all')
dataset.head()

,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
0,-15.138926,2.106980,2.735102,3.355777,0.759586,-19.430781,5.612832,3.089403,4.227920,11.048449,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-15.428091,16.758788,5.130048,4.540909,18.783533,-18.542018,4.338640,2.660826,9.064463,17.644304,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-2.155112,-15.990361,3.499145,2.696861,6.662967,-6.572656,4.626157,3.962111,-15.191983,15.354067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.176409,12.268334,2.825052,2.235162,-8.728322,-7.598213,2.592677,3.158372,19.831005,-14.813372,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,15.039568,-3.857973,3.553933,2.890238,3.239212,-1.239263,5.175412,3.439370,-5.067780,0.809135,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
